In [1]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import pandas as pd
from datasets import load_dataset

# Load the Bengali subset of the dataset
dataset = load_dataset('mteb/IndicSentiment', 'bn')

# Convert to pandas dataframe
df = dataset['train'].to_pandas()  # Assuming you want the training split

# Display the dataframe
df.head()

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/156 [00:00<?, ? examples/s]

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang
0,Home,Appliances,Home theater,Soundbars,Boat,"Bluetooth/wireless, HDMI, audio output mode, i...",HDMI,This boat's soundbar is still wire-connectivit...,Negative,এই বোটের সাউন্ডবারটি এখনও সব স্পিকারের জন্য তা...,bn
1,Hobbies,Music,Audio Output,headphones,Zeb Paradise,"on-ear, in-ear, wired, bluetooth, earbuds, noi...",Over-ear with mic,Foldable type of microphone with mic and micro...,Positive,মাইক ও মাইক্রো এসডি কার্ড স্লটসহ ভাঁজযোগ্য ধরন...,bn
2,Entertainment,Apps,Social Media,Social networking,Instagram,"find friends, share photos and moments, free m...",daily status,The recently included feature of stories by de...,Positive,24 ঘণ্টার জন্য গল্পের যে বৈশিষ্ট্যটি ডিফল্টভাব...,bn
3,Transportation,Air,Flights,International,Emirates,"luggage allowance, affordable rates, luxury, f...",Rates Luggage allowance,"Rates are competitive, almost always the best ...",Positive,"""দরগুলি প্রতিযোগিতামূলক, প্রায় সবসময় বাজারে ...",bn
4,Home,Appliances,Fan,Exhaust fan,Bajaj Maxima,"remove moisture/unpleasant odour, air delivery...",Front Shutter,Looks very big and efficient. But since there ...,Negative,দেখতে অনেক বড় এবং দক্ষ। কিন্তু যেহেতু সামনে কো...,bn


In [3]:
zero_shot = """আপনাকে একটি পণ্যের রিভিউ দেওয়া হয়েছে। আপনাকে রিভিউটির অনুভূতি নির্ধারণ করতে হবে, যা পজিটিভ বা নেগেটিভ হতে পারে।

"""

### Three-shot examples

We took the examples from IndicSentiment dataset (not in the eval. / test split).

In [4]:
three_shot = """আপনাকে একটি পণ্যের রিভিউ দেওয়া হয়েছে। আপনাকে রিভিউটির অনুভূতি নির্ধারণ করতে হবে, যা পজিটিভ বা নেগেটিভ হতে পারে।

### উদাহরণ ১
### রিভিউ:
দরগুলি প্রতিযোগিতামূলক, প্রায় সবসময় বাজারে সেরা।

### অনুভূতি:
পজিটিভ

### উদাহরণ ২
### রিভিউ:
অন কল কানেক্টিভিটি অনেক সময় খুব কম হয়।

### অনুভূতি:
নেগেটিভ

### উদাহরণ ৩
### রিভিউ:
হুক এবং লুপ ডিজাইন সহজেই সেট আপ এবং বহন করা যায়।

### অনুভূতি:
পজিটিভ

### আপনার কাজ

"""

In [5]:
chain_of_thought = """আপনাকে একটি পণ্যের রিভিউ দেওয়া হয়েছে। আপনাকে রিভিউটির অনুভূতি নির্ধারণ করতে হবে, যা পজিটিভ বা নেগেটিভ হতে পারে। রিভিউটির অনুভূতি নির্ধারণ করতে গিয়ে নিম্নলিখিত ধাপগুলি অনুসরণ করুন:

### ধাপ ১: রিভিউ পড়া
রিভিউটি পড়ুন এবং এতে কী বলা হয়েছে তা বোঝার চেষ্টা করুন।

### ধাপ ২: ইতিবাচক বা নেতিবাচক শব্দ চিহ্নিত করা
রিভিউতে ব্যবহৃত ইতিবাচক (ভালো, সন্তুষ্ট, দুর্দান্ত) বা নেতিবাচক (খারাপ, বাজে, নিম্নমানের) শব্দগুলি চিহ্নিত করুন।

### ধাপ ৩: অনুভূতি নির্ধারণ
রিভিউতে ইতিবাচক শব্দ বেশি হলে পজিটিভ এবং নেতিবাচক শব্দ বেশি হলে নেগেটিভ অনুভূতি নির্ধারণ করুন।

### উদাহরণ ১
### রিভিউ:
দরগুলি প্রতিযোগিতামূলক, প্রায় সবসময় বাজারে সেরা।

#### চিন্তাধারা:
- ইতিবাচক শব্দ: প্রতিযোগিতামূলক, সেরা
- অনুভূতি: পজিটিভ

### অনুভূতি:
পজিটিভ

### উদাহরণ ২
### রিভিউ:
অন কল কানেক্টিভিটি অনেক সময় খুব কম হয়।

#### চিন্তাধারা:
- নেতিবাচক শব্দ: অনেক সময় খুব কম
- অনুভূতি: নেগেটিভ

### অনুভূতি:
নেগেটিভ

### উদাহরণ ৩
### রিভিউ:
হুক এবং লুপ ডিজাইন সহজেই সেট আপ এবং বহন করা যায়।

#### চিন্তাধারা:
- ইতিবাচক শব্দ: সহজেই সেট আপ এবং বহন করা যায়
- অনুভূতি: পজিটিভ

### অনুভূতি:
পজিটিভ

### আপনার কাজ


"""

## GPT-3.5-Turbo-0125

In [6]:
%pip install --upgrade openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 404.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [7]:
from openai import OpenAI
import os

## Set the API key and model name
MODEL="gpt-3.5-turbo-0125"

from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [8]:
def get_gpt3_5_turbo_output(prompt = "", max_out_tokens = 128, temp = 0, top_p = 0.9):

    response = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt},
      ],
      temperature = temp,
      max_tokens = max_out_tokens,
      top_p = top_p
    )

    prompt_response = " " + response.choices[0].message.content
    return prompt_response

In [9]:
get_gpt3_5_turbo_output("বাংলাদেশের ৫ জন কবির নাম লিস্ট আকারে লিখ।")

' 1. কাজী নজরুল ইসলাম\n2. জিবনানন্দ দাশ\n3. রবীন্দ্রনাথ ঠাকুর\n4. মিজানুর রহমান\n5. শামসুর রাহমান'

In [10]:
category_translation = {
    'Positive': 'পজিটিভ',
    'Negative': 'নেগেটিভ',
}

### Zero-shot

In [11]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gpt3_5_turbo_zs_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = zero_shot + " ###\nরিভিউ: " + review + "\n### অনুভূতি:"

    try:
        df.at[i, 'predicted_category'] = get_gpt3_5_turbo_output(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gpt3_5_turbo_zs_sentiment_results.parquet')

# Save final progress
df.to_parquet('gpt3_5_turbo_zs_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

In [13]:
zs_results = pd.read_parquet("gpt3_5_turbo_zs_sentiment_results.parquet")
zs_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
151,Home,Appliances,Home theater,Home theater systems,iKall,"Bluetooth, USB &HDMI, Dolby, voice control, sp...",Dolby output,IKall has now launched a new home theater syst...,Positive,আইকল এখন ডলবি আউটপুট সহ একটি নতুন হোম থিয়েটার ...,bn,এই রিভিউটি দেখে মনে হচ্ছে যে আইকলের এই নতুন হ...
152,Home,Appliances,Home theater,Tower speakers,iKall,"speaker connectivity, speaker feature, wattage...",Wattage,iKall is giving two 500 Watts speakers in its ...,Negative,আইকল তাদের টাওয়ার স্পিকার সেটে 500 ওয়াটের দুটি...,bn,এই রিভিউটি পজিটিভ এবং নেগেটিভ দুটি দিক উল্লেখ...
153,Health/Wellness,Living,Stay/ Experience,Resorts,Vedic Village Spa Resort (Kolkata),"Swimming pool, fitness center, parking, wifi, ...","Wifi, air conditioning, family friendly","The Resort doesn't have wifi, so you have to b...",Negative,"রিসোর্টে ওয়াইফাই নেই, তাই আপনাকে আপনার ফোন ইন...",bn,"এই রিভিউটি পজিটিভ নয়, কারণ রিসোর্টে ওয়াইফাই..."
154,Entertainment,Movies,Genres,Tragedy,The Tunnel,"serious, storyline, performances, emotional, m...",Peformances and Moving,"Well directed, & acted, & excellent cinematogr...",Positive,"ভালো পরিচালনা এবং অভিনয়, একটি অসাধারণ সিনেমাটো...",bn,ধন্যবাদ রিভিউটি শুনানোর জন্য। এই পণ্যটির পরিচ...
155,Fashion,Beauty,Fragrance,Deodorants for women,Dove Advanced Care Antiperspirant Deodorant Se...,"alcohol free, skin whitening, odour control, l...",Long lasting,Its a terrible product!! Works only for a few ...,Negative,এটা একটা খারাপ প্রোডাক্ট!! মাত্র কয়েক ঘণ্টা কা...,bn,এই রিভিউটি নেগেটিভ হতে পারে কারণ এটা প্রোডাক্...


### Three Shot

In [14]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gpt3_5_turbo_ts_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = three_shot + " ###\nরিভিউ: " + review + "\n### অনুভূতি:"

    try:
        df.at[i, 'predicted_category'] = get_gpt3_5_turbo_output(prompt_input, max_out_tokens = 256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gpt3_5_turbo_ts_sentiment_results.parquet')

# Save final progress
df.to_parquet('gpt3_5_turbo_ts_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

In [18]:
ts_results = pd.read_parquet("gpt3_5_turbo_ts_sentiment_results.parquet")
ts_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
151,Home,Appliances,Home theater,Home theater systems,iKall,"Bluetooth, USB &HDMI, Dolby, voice control, sp...",Dolby output,IKall has now launched a new home theater syst...,Positive,আইকল এখন ডলবি আউটপুট সহ একটি নতুন হোম থিয়েটার ...,bn,পজিটিভ
152,Home,Appliances,Home theater,Tower speakers,iKall,"speaker connectivity, speaker feature, wattage...",Wattage,iKall is giving two 500 Watts speakers in its ...,Negative,আইকল তাদের টাওয়ার স্পিকার সেটে 500 ওয়াটের দুটি...,bn,নেগেটিভ
153,Health/Wellness,Living,Stay/ Experience,Resorts,Vedic Village Spa Resort (Kolkata),"Swimming pool, fitness center, parking, wifi, ...","Wifi, air conditioning, family friendly","The Resort doesn't have wifi, so you have to b...",Negative,"রিসোর্টে ওয়াইফাই নেই, তাই আপনাকে আপনার ফোন ইন...",bn,নেগেটিভ
154,Entertainment,Movies,Genres,Tragedy,The Tunnel,"serious, storyline, performances, emotional, m...",Peformances and Moving,"Well directed, & acted, & excellent cinematogr...",Positive,"ভালো পরিচালনা এবং অভিনয়, একটি অসাধারণ সিনেমাটো...",bn,পজিটিভ
155,Fashion,Beauty,Fragrance,Deodorants for women,Dove Advanced Care Antiperspirant Deodorant Se...,"alcohol free, skin whitening, odour control, l...",Long lasting,Its a terrible product!! Works only for a few ...,Negative,এটা একটা খারাপ প্রোডাক্ট!! মাত্র কয়েক ঘণ্টা কা...,bn,নেগেটিভ


### Chain of Thought

In [16]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gpt3_5_turbo_cot_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = chain_of_thought + "\n### রিভিউ: " + review

    try:
        df.at[i, 'predicted_category'] = get_gpt3_5_turbo_output(prompt_input, max_out_tokens = 256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gpt3_5_turbo_cot_sentiment_results.parquet')

# Save final progress
df.to_parquet('gpt3_5_turbo_cot_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

In [ ]:
# Display all columns
pd.set_option('display.max_columns', None)

# Display all rows
pd.set_option('display.max_rows', None)

# Display full text in each cell
pd.set_option('display.max_colwidth', None)

# If you have a large DataFrame and want to see it in a scrollable window
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [17]:
cot_results = pd.read_parquet("gpt3_5_turbo_cot_sentiment_results.parquet")
cot_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
151,Home,Appliances,Home theater,Home theater systems,iKall,"Bluetooth, USB &HDMI, Dolby, voice control, sp...",Dolby output,IKall has now launched a new home theater syst...,Positive,আইকল এখন ডলবি আউটপুট সহ একটি নতুন হোম থিয়েটার ...,bn,### ধাপ ১: রিভিউ পড়া\nরিভিউটি পড়া হয়েছে এব...
152,Home,Appliances,Home theater,Tower speakers,iKall,"speaker connectivity, speaker feature, wattage...",Wattage,iKall is giving two 500 Watts speakers in its ...,Negative,আইকল তাদের টাওয়ার স্পিকার সেটে 500 ওয়াটের দুটি...,bn,### ধাপ ১: রিভিউ পড়া\nরিভিউটি পড়া হয়েছে। এ...
153,Health/Wellness,Living,Stay/ Experience,Resorts,Vedic Village Spa Resort (Kolkata),"Swimming pool, fitness center, parking, wifi, ...","Wifi, air conditioning, family friendly","The Resort doesn't have wifi, so you have to b...",Negative,"রিসোর্টে ওয়াইফাই নেই, তাই আপনাকে আপনার ফোন ইন...",bn,### ধাপ ১: রিভিউ পড়া\nরিভিউটি পড়া হয়েছে এব...
154,Entertainment,Movies,Genres,Tragedy,The Tunnel,"serious, storyline, performances, emotional, m...",Peformances and Moving,"Well directed, & acted, & excellent cinematogr...",Positive,"ভালো পরিচালনা এবং অভিনয়, একটি অসাধারণ সিনেমাটো...",bn,### ধাপ ১: রিভিউ পড়া\nরিভিউটি পড়া হয়েছে এব...
155,Fashion,Beauty,Fragrance,Deodorants for women,Dove Advanced Care Antiperspirant Deodorant Se...,"alcohol free, skin whitening, odour control, l...",Long lasting,Its a terrible product!! Works only for a few ...,Negative,এটা একটা খারাপ প্রোডাক্ট!! মাত্র কয়েক ঘণ্টা কা...,bn,### রিভিউ:\nএটা একটা খারাপ প্রোডাক্ট!! মাত্র ...


### Category Prediction

In [19]:
def clean_predicted_category(text, category_translation):
    # Check if the text is None
    if text is None:
        return ""

    # Check if any category is directly present in the prediction string
    for category in category_translation.values():
        if category in text:
            return category

    # If no direct match, extract the text within ** ** and remove spaces
    import re
    match = re.search(r'\*\*([^*]*)\*\*', text)
    if match:
        cleaned_text = match.group(1).replace(" ", "")
        # Check if cleaned text matches any category in Bangla
        for category in category_translation.values():
            if cleaned_text == category:
                return category

    # If no match found, return None
    return ""

In [20]:
clean_predicted_category("অনুভূতি: **নেগেটিভ** \n\nরিভিউতে বলা হয়েছে ...", category_translation)

'নেগেটিভ'

In [21]:
clean_predicted_category("অনুভূতি: **পজিটিভ** \n\nদরগুলি প্রতিযোগিতামূলক...", category_translation)

'পজিটিভ'

In [22]:
clean_predicted_category(None, category_translation)

''

### Classification Evaluation

In [23]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelBinarizer

import numpy as np

def update_cleaned_predicted_category(row):
    if pd.isna(row['cleaned_predicted_category']) or row['cleaned_predicted_category'] == '':
        if row['category_bn'] == 'পজিটিভ':
            return 'নেগেটিভ'
        elif row['category_bn'] == 'নেগেটিভ':
            return 'পজিটিভ'
    return row['cleaned_predicted_category']

def evaluate_classification(df):
    df['category'] = df['LABEL']
    # Map English category to Bangla
    df['category_bn'] = df['category'].map(category_translation)

    # Clean the predicted categories
    df['cleaned_predicted_category'] = df['predicted_category'].apply(clean_predicted_category, args=(category_translation,))

    # Update the cleaned predicted categories
    for index, row in df.iterrows():
        old_value = row['cleaned_predicted_category']
        if pd.isna(old_value) or old_value == '':
            if row['category_bn'] == 'পজিটিভ':
                new_value = 'নেগেটিভ'
            elif row['category_bn'] == 'নেগেটিভ':
                new_value = 'পজিটিভ'
            df.at[index, 'cleaned_predicted_category'] = new_value

    unique_labels = set(df['category_bn'].unique()).union(set(df['cleaned_predicted_category'].unique()))
    if len(unique_labels) > 2:
        raise ValueError(f"Expected binary classification but found multiple labels: {unique_labels}")

    # Calculate metrics
    y_true = df['category_bn']
    y_pred = df['cleaned_predicted_category']


    # Ensure the labels are binary
    lb = LabelBinarizer()
    y_true_binarized = lb.fit_transform(y_true)
    y_pred_binarized = lb.transform(y_pred)

    if len(lb.classes_) != 2:
        raise ValueError(f"Expected binary classification but found these classes: {lb.classes_}")

    pos_label = lb.classes_[1]  # The positive label

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    recall = recall_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    f1 = f1_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    auc_roc = roc_auc_score(y_true_binarized, y_pred_binarized)
    logloss = log_loss(y_true_binarized, y_pred_binarized)

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'log_loss': logloss
    }

    return metrics

In [24]:
zs_df = pd.read_parquet("gpt3_5_turbo_zs_sentiment_results.parquet")
metrics = evaluate_classification(zs_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.3654
precision: 0.3462
recall: 0.3600
f1_score: 0.3529
auc_roc: 0.3652
log_loss: 22.8739


In [25]:
ts_df = pd.read_parquet("gpt3_5_turbo_ts_sentiment_results.parquet")
metrics = evaluate_classification(ts_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.9295
precision: 0.9706
recall: 0.8800
f1_score: 0.9231
auc_roc: 0.9277
log_loss: 2.5415


In [26]:
cot_df = pd.read_parquet("gpt3_5_turbo_cot_sentiment_results.parquet")
metrics = evaluate_classification(cot_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.0192
precision: 0.0125
recall: 0.0133
f1_score: 0.0129
auc_roc: 0.0190
log_loss: 35.3505


## GPT-4o

In [ ]:
%pip install --upgrade openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 404.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [27]:
from openai import OpenAI
import os

## Set the API key and model name
MODEL="gpt-4o"

from google.colab import userdata

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [28]:
def get_gpt4omni_output(prompt = "", max_out_tokens = 128, temp = 0, top_p = 0.9):

    response = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt},
      ],
      temperature = temp,
      max_tokens = max_out_tokens,
      top_p = top_p
    )

    prompt_response = " " + response.choices[0].message.content
    return prompt_response

In [29]:
get_gpt4omni_output("বাংলাদেশের ৫ জন কবির নাম লিস্ট আকারে লিখ।")

' নিশ্চিতভাবে, এখানে বাংলাদেশের ৫ জন প্রখ্যাত কবির নামের একটি তালিকা দেওয়া হলো:\n\n1. কাজী নজরুল ইসলাম\n2. জসীম উদ্দীন\n3. শামসুর রাহমান\n4. সুনীল গঙ্গোপাধ্যায়\n5. আল মাহমুদ'

In [30]:
category_translation = {
    'Positive': 'পজিটিভ',
    'Negative': 'নেগেটিভ',
}

### Zero-shot

In [31]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gpt4omni_zs_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = zero_shot + " ###\nরিভিউ: " + review + "\n### অনুভূতি:"

    try:
        df.at[i, 'predicted_category'] = get_gpt4omni_output(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gpt4omni_zs_sentiment_results.parquet')

# Save final progress
df.to_parquet('gpt4omni_zs_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

In [32]:
zs_results = pd.read_parquet("gpt4omni_zs_sentiment_results.parquet")
zs_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
151,Home,Appliances,Home theater,Home theater systems,iKall,"Bluetooth, USB &HDMI, Dolby, voice control, sp...",Dolby output,IKall has now launched a new home theater syst...,Positive,আইকল এখন ডলবি আউটপুট সহ একটি নতুন হোম থিয়েটার ...,bn,পজিটিভ
152,Home,Appliances,Home theater,Tower speakers,iKall,"speaker connectivity, speaker feature, wattage...",Wattage,iKall is giving two 500 Watts speakers in its ...,Negative,আইকল তাদের টাওয়ার স্পিকার সেটে 500 ওয়াটের দুটি...,bn,নেগেটিভ
153,Health/Wellness,Living,Stay/ Experience,Resorts,Vedic Village Spa Resort (Kolkata),"Swimming pool, fitness center, parking, wifi, ...","Wifi, air conditioning, family friendly","The Resort doesn't have wifi, so you have to b...",Negative,"রিসোর্টে ওয়াইফাই নেই, তাই আপনাকে আপনার ফোন ইন...",bn,নেগেটিভ
154,Entertainment,Movies,Genres,Tragedy,The Tunnel,"serious, storyline, performances, emotional, m...",Peformances and Moving,"Well directed, & acted, & excellent cinematogr...",Positive,"ভালো পরিচালনা এবং অভিনয়, একটি অসাধারণ সিনেমাটো...",bn,পজিটিভ
155,Fashion,Beauty,Fragrance,Deodorants for women,Dove Advanced Care Antiperspirant Deodorant Se...,"alcohol free, skin whitening, odour control, l...",Long lasting,Its a terrible product!! Works only for a few ...,Negative,এটা একটা খারাপ প্রোডাক্ট!! মাত্র কয়েক ঘণ্টা কা...,bn,### অনুভূতি: নেগেটিভ


### Three Shot

In [33]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gpt4omni_ts_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = three_shot + " ###\nরিভিউ: " + review + "\n### অনুভূতি:"

    try:
        df.at[i, 'predicted_category'] = get_gpt4omni_output(prompt_input, max_out_tokens = 256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gpt4omni_ts_sentiment_results.parquet')

# Save final progress
df.to_parquet('gpt4omni_ts_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

In [34]:
ts_results = pd.read_parquet("gpt4omni_ts_sentiment_results.parquet")
ts_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
151,Home,Appliances,Home theater,Home theater systems,iKall,"Bluetooth, USB &HDMI, Dolby, voice control, sp...",Dolby output,IKall has now launched a new home theater syst...,Positive,আইকল এখন ডলবি আউটপুট সহ একটি নতুন হোম থিয়েটার ...,bn,পজিটিভ
152,Home,Appliances,Home theater,Tower speakers,iKall,"speaker connectivity, speaker feature, wattage...",Wattage,iKall is giving two 500 Watts speakers in its ...,Negative,আইকল তাদের টাওয়ার স্পিকার সেটে 500 ওয়াটের দুটি...,bn,### অনুভূতি:\nনেগেটিভ
153,Health/Wellness,Living,Stay/ Experience,Resorts,Vedic Village Spa Resort (Kolkata),"Swimming pool, fitness center, parking, wifi, ...","Wifi, air conditioning, family friendly","The Resort doesn't have wifi, so you have to b...",Negative,"রিসোর্টে ওয়াইফাই নেই, তাই আপনাকে আপনার ফোন ইন...",bn,নেগেটিভ
154,Entertainment,Movies,Genres,Tragedy,The Tunnel,"serious, storyline, performances, emotional, m...",Peformances and Moving,"Well directed, & acted, & excellent cinematogr...",Positive,"ভালো পরিচালনা এবং অভিনয়, একটি অসাধারণ সিনেমাটো...",bn,পজিটিভ
155,Fashion,Beauty,Fragrance,Deodorants for women,Dove Advanced Care Antiperspirant Deodorant Se...,"alcohol free, skin whitening, odour control, l...",Long lasting,Its a terrible product!! Works only for a few ...,Negative,এটা একটা খারাপ প্রোডাক্ট!! মাত্র কয়েক ঘণ্টা কা...,bn,নেগেটিভ


### Chain of Thought

In [35]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gpt4omni_cot_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = chain_of_thought + "\n### রিভিউ: " + review

    try:
        df.at[i, 'predicted_category'] = get_gpt4omni_output(prompt_input, max_out_tokens = 256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gpt4omni_cot_sentiment_results.parquet')

# Save final progress
df.to_parquet('gpt4omni_cot_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

In [ ]:
# Display all columns
pd.set_option('display.max_columns', None)

# Display all rows
pd.set_option('display.max_rows', None)

# Display full text in each cell
pd.set_option('display.max_colwidth', None)

# If you have a large DataFrame and want to see it in a scrollable window
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [36]:
cot_results = pd.read_parquet("gpt4omni_cot_sentiment_results.parquet")
cot_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
151,Home,Appliances,Home theater,Home theater systems,iKall,"Bluetooth, USB &HDMI, Dolby, voice control, sp...",Dolby output,IKall has now launched a new home theater syst...,Positive,আইকল এখন ডলবি আউটপুট সহ একটি নতুন হোম থিয়েটার ...,bn,### ধাপ ১: রিভিউ পড়া\nআইকল এখন ডলবি আউটপুট স...
152,Home,Appliances,Home theater,Tower speakers,iKall,"speaker connectivity, speaker feature, wattage...",Wattage,iKall is giving two 500 Watts speakers in its ...,Negative,আইকল তাদের টাওয়ার স্পিকার সেটে 500 ওয়াটের দুটি...,bn,### ধাপ ১: রিভিউ পড়া\nরিভিউটি পড়া হয়েছে এব...
153,Health/Wellness,Living,Stay/ Experience,Resorts,Vedic Village Spa Resort (Kolkata),"Swimming pool, fitness center, parking, wifi, ...","Wifi, air conditioning, family friendly","The Resort doesn't have wifi, so you have to b...",Negative,"রিসোর্টে ওয়াইফাই নেই, তাই আপনাকে আপনার ফোন ইন...",bn,### ধাপ ১: রিভিউ পড়া\nরিভিউটি পড়ে বোঝা যাচ্...
154,Entertainment,Movies,Genres,Tragedy,The Tunnel,"serious, storyline, performances, emotional, m...",Peformances and Moving,"Well directed, & acted, & excellent cinematogr...",Positive,"ভালো পরিচালনা এবং অভিনয়, একটি অসাধারণ সিনেমাটো...",bn,### ধাপ ১: রিভিউ পড়া\nরিভিউ: ভালো পরিচালনা এ...
155,Fashion,Beauty,Fragrance,Deodorants for women,Dove Advanced Care Antiperspirant Deodorant Se...,"alcohol free, skin whitening, odour control, l...",Long lasting,Its a terrible product!! Works only for a few ...,Negative,এটা একটা খারাপ প্রোডাক্ট!! মাত্র কয়েক ঘণ্টা কা...,bn,### ধাপ ১: রিভিউ পড়া\nরিভিউটি পড়েছি এবং এতে...


### Category Prediction

In [38]:
def clean_predicted_category(text, category_translation):
    # Check if the text is None
    if text is None:
        return ""

    # Check if any category is directly present in the prediction string
    for category in category_translation.values():
        if category in text:
            return category

    # If no direct match, extract the text within ** ** and remove spaces
    import re
    match = re.search(r'\*\*([^*]*)\*\*', text)
    if match:
        cleaned_text = match.group(1).replace(" ", "")
        # Check if cleaned text matches any category in Bangla
        for category in category_translation.values():
            if cleaned_text == category:
                return category

    # If no match found, return None
    return ""

In [39]:
clean_predicted_category("অনুভূতি: **নেগেটিভ** \n\nরিভিউতে বলা হয়েছে ...", category_translation)

'নেগেটিভ'

In [40]:
clean_predicted_category("অনুভূতি: **পজিটিভ** \n\nদরগুলি প্রতিযোগিতামূলক...", category_translation)

'পজিটিভ'

In [41]:
clean_predicted_category(None, category_translation)

''

### Classification Evaluation

In [42]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelBinarizer

import numpy as np

def update_cleaned_predicted_category(row):
    if pd.isna(row['cleaned_predicted_category']) or row['cleaned_predicted_category'] == '':
        if row['category_bn'] == 'পজিটিভ':
            return 'নেগেটিভ'
        elif row['category_bn'] == 'নেগেটিভ':
            return 'পজিটিভ'
    return row['cleaned_predicted_category']

def evaluate_classification(df):
    df['category'] = df['LABEL']
    # Map English category to Bangla
    df['category_bn'] = df['category'].map(category_translation)

    # Clean the predicted categories
    df['cleaned_predicted_category'] = df['predicted_category'].apply(clean_predicted_category, args=(category_translation,))

    # Update the cleaned predicted categories
    for index, row in df.iterrows():
        old_value = row['cleaned_predicted_category']
        if pd.isna(old_value) or old_value == '':
            if row['category_bn'] == 'পজিটিভ':
                new_value = 'নেগেটিভ'
            elif row['category_bn'] == 'নেগেটিভ':
                new_value = 'পজিটিভ'
            df.at[index, 'cleaned_predicted_category'] = new_value

    unique_labels = set(df['category_bn'].unique()).union(set(df['cleaned_predicted_category'].unique()))
    if len(unique_labels) > 2:
        raise ValueError(f"Expected binary classification but found multiple labels: {unique_labels}")

    # Calculate metrics
    y_true = df['category_bn']
    y_pred = df['cleaned_predicted_category']


    # Ensure the labels are binary
    lb = LabelBinarizer()
    y_true_binarized = lb.fit_transform(y_true)
    y_pred_binarized = lb.transform(y_pred)

    if len(lb.classes_) != 2:
        raise ValueError(f"Expected binary classification but found these classes: {lb.classes_}")

    pos_label = lb.classes_[1]  # The positive label

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    recall = recall_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    f1 = f1_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    auc_roc = roc_auc_score(y_true_binarized, y_pred_binarized)
    logloss = log_loss(y_true_binarized, y_pred_binarized)

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'log_loss': logloss
    }

    return metrics

In [43]:
zs_df = pd.read_parquet("gpt4omni_zs_sentiment_results.parquet")
metrics = evaluate_classification(zs_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.9551
precision: 0.9857
recall: 0.9200
f1_score: 0.9517
auc_roc: 0.9538
log_loss: 1.6173


In [44]:
ts_df = pd.read_parquet("gpt4omni_ts_sentiment_results.parquet")
metrics = evaluate_classification(ts_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.9744
precision: 0.9863
recall: 0.9600
f1_score: 0.9730
auc_roc: 0.9738
log_loss: 0.9242


In [45]:
cot_df = pd.read_parquet("gpt4omni_cot_sentiment_results.parquet")
metrics = evaluate_classification(cot_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.9872
precision: 0.9867
recall: 0.9867
f1_score: 0.9867
auc_roc: 0.9872
log_loss: 0.4621
